In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
order_items = pd.read_parquet('../dataset/Looker Ecommerce BigQuery Dataset/Optimization/order_items_optimized.parquet')

In [3]:
# 예측확인용으로 날짜 필터링해서 일부 남겨두기 용
use_standard_date = pd.to_datetime('2023-12-01 00:00:00.000000')
use_standard_date = use_standard_date.tz_localize('UTC')
order_items_extracted = order_items[order_items['created_at'] < use_standard_date]

# 구매건만 남기기
order_items_extracted = order_items_extracted[(order_items_extracted['status']!="Returned") & (order_items_extracted['status'] != "Cancelled")]
order_items_extracted.head(3)

,order_item_id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,0.02
5,154022,106052,84693,14235,415780,Shipped,2023-09-19 09:13:08+00:00,2023-09-16 09:24:00+00:00,NaT,NaT,0.02


# CLV T넣어서 lifetime 패키지 사용해서 하는 방식

https://medium.com/analytics-vidhya/customer-life-time-value-prediction-by-using-bg-nbd-gamma-gamma-models-and-applied-example-in-997a5ee481ad

In [6]:
import datetime as dt

from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

In [ ]:
# rfm = order_items_extracted.groupby('user_id').agg(
#     recency=('created_at', lambda date: (use_standard_date - date.max()).days), 
#     frequency=('order_id','nunique'), 
#     monetary=('sale_price', 'sum')
# )
# rfm

In [8]:
cltv = order_items_extracted.groupby('user_id').agg(
    recency=('created_at', lambda date: (use_standard_date - date.max()).days), 
    T=('created_at', lambda date: (use_standard_date - date.min()).days),
    frequency=('order_id','nunique'), 
    monetary=('sale_price', 'sum')
)
cltv

,recency,T,frequency,monetary
user_id,,,,
1,498,499,1,159.990000
2,648,648,1,22.000000
3,114,265,4,402.200001
4,408,408,1,148.000000
6,598,599,1,144.820004
...,...,...,...,...
99989,90,94,1,15.120000
99992,478,479,1,55.950001
99996,260,1216,2,123.870001


In [12]:
cltv['monetary'] = cltv['monetary'] / cltv['frequency']

In [13]:
# transforming days to weeks
cltv['recency'] = cltv['recency'] / 7
cltv['T'] = cltv['T'] / 7

In [16]:
print(len(cltv['frequency']))
print(len(cltv[cltv['frequency']>1]))

58464
16834


In [17]:
cltv = cltv[cltv['frequency']>1]

In [18]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv['frequency'], cltv['recency'], cltv['T'])

<lifetimes.BetaGeoFitter: fitted with 16834 subjects, a: 2.73, alpha: 10.42, b: 5.46, r: 1.17>

In [19]:
bgf.conditional_expected_number_of_purchases_up_to_time(1, # week
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['T']).sort_values(ascending=False).head(10)

user_id
22606    0.222803
19606    0.218084
59558    0.213825
41852    0.209877
31092    0.195131
65615    0.193016
91919    0.192720
30136    0.192531
3398     0.191453
40976    0.190188
dtype: float64

In [20]:
bgf.conditional_expected_number_of_purchases_up_to_time(4, 
                                                 # 4 weeks = 1 month
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['T']).sort_values(ascending=False).head(10)

user_id
22606    0.771274
19606    0.759522
59558    0.746810
41852    0.728777
65615    0.683464
3398     0.680875
31092    0.679861
91919    0.672546
30136    0.670804
40976    0.663710
dtype: float64

In [21]:
bgf.conditional_expected_number_of_purchases_up_to_time(4 * 6, 
                                           # weeks * count = months
                                                        cltv['frequency'],
                                                        cltv['recency'],
                                                        cltv['T']).sort_values(ascending=False).head(10)

user_id
19606    2.430457
22606    2.421183
59558    2.411757
3398     2.336631
65615    2.310652
41852    2.310161
31092    2.195655
91919    2.183114
30136    2.166402
40976    2.154434
dtype: float64

In [22]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv['frequency'], cltv['monetary'])

<lifetimes.GammaGammaFitter: fitted with 16834 subjects, p: 4.14, q: 0.62, v: 3.99>

In [23]:
ggf.conditional_expected_average_profit(cltv['frequency'],
                               cltv['monetary']).sort_values(ascending=False).head(10)

user_id
6878     681.269713
58880    680.708998
11609    671.506983
22656    664.988018
9607     647.144700
76455    626.738867
89026    616.808444
78981    609.435306
93565    606.359233
8266     598.409657
dtype: float64

In [24]:
# The customers' lifetime values expected to in the next 3 months
cltv['cltv_pred_3_months'] = ggf.customer_lifetime_value(bgf,
                                   cltv['frequency'],
                                   cltv['recency'],
                                   cltv['T'],
                                   cltv['monetary'],
                                   time=3,  # 3 months
                                   freq="W",  
# frequency information of T. In this case we set week by using 'W'
                                   discount_rate=0.01)
cltv

,recency,T,frequency,monetary,cltv_pred_3_months
user_id,,,,,
3,16.285714,37.857143,4,100.550000,15.182954
12,22.142857,33.714286,3,99.190000,45.862721
17,160.714286,232.428571,3,249.329995,21.298199
18,13.285714,21.285714,2,39.000000,21.664473
29,68.714286,193.000000,2,34.495000,0.779711
...,...,...,...,...,...
99969,2.142857,14.571429,3,56.976667,12.967643
99979,12.571429,12.714286,2,75.134998,75.356068
99983,13.857143,89.428571,4,126.474999,0.163712


In [25]:
cltv['segment'] = pd.qcut(cltv['cltv_pred_3_months'],4,['D','C','B','A'])

cltv

,recency,T,frequency,monetary,cltv_pred_3_months,segment
user_id,,,,,,
3,16.285714,37.857143,4,100.550000,15.182954,B
12,22.142857,33.714286,3,99.190000,45.862721,A
17,160.714286,232.428571,3,249.329995,21.298199,A
18,13.285714,21.285714,2,39.000000,21.664473,A
29,68.714286,193.000000,2,34.495000,0.779711,D
...,...,...,...,...,...,...
99969,2.142857,14.571429,3,56.976667,12.967643,B
99979,12.571429,12.714286,2,75.134998,75.356068,A
99983,13.857143,89.428571,4,126.474999,0.163712,D


In [26]:
# The customers' lifetime values expected to in the next 3 months
cltv['cltv_pred_12_months'] = ggf.customer_lifetime_value(bgf,
                                   cltv['frequency'],
                                   cltv['recency'],
                                   cltv['T'],
                                   cltv['monetary'],
                                   time=12,  # 12 months
                                   freq="W",  
# frequency information of T. In this case we set week by using 'W'
                                   discount_rate=0.01)
cltv

,recency,T,frequency,monetary,cltv_pred_3_months,segment,cltv_pred_12_months
user_id,,,,,,,
3,16.285714,37.857143,4,100.550000,15.182954,B,39.618930
12,22.142857,33.714286,3,99.190000,45.862721,A,120.157652
17,160.714286,232.428571,3,249.329995,21.298199,A,74.382965
18,13.285714,21.285714,2,39.000000,21.664473,A,53.904059
29,68.714286,193.000000,2,34.495000,0.779711,D,2.711206
...,...,...,...,...,...,...,...
99969,2.142857,14.571429,3,56.976667,12.967643,B,28.684845
99979,12.571429,12.714286,2,75.134998,75.356068,A,170.688340
99983,13.857143,89.428571,4,126.474999,0.163712,D,0.503722


# 가장 일반적으로 쓰는 방식

고객 가치 = 평균 구매 가치 x 평균 구매 빈도
          = 5달러 x 4번 = 20달러/월

CLV = 고객 가치 x 고객 평균 수명
    = 20달러/월 x 12개월 x 20년
    = 4,800달러

평균 구매 가치(Average Purchase Value, APV): 이는 고객이 한 번 구매할 때 평균적으로 지출하는 금액입니다. 평균 구매 가치를 구하기 위해서는 특정 기간 동안의 총 매출을 해당 기간 동안의 총 구매 횟수로 나눕니다.
평균 구매 빈도(Average Purchase Frequency Rate, APFR): 이는 고객이 특정 기간 동안 얼마나 자주 구매하는지를 나타냅니다. 평균 구매 빈도를 구하기 위해서는 특정 기간 동안의 총 구매 횟수를 구매한 고객의 총 수로 나눕니다.
평균 고객 가치(Average Customer Value, ACV): 평균 구매 가치와 평균 구매 빈도를 곱하여 계산할 수 있습니다.

고객 평균 수명(Customer Lifespan): 이는 고객이 기업과 거래를 지속하는 평균 기간을 의미합니다. 고객 평균 수명을 구하는 정확한 공식은 업계마다 다를 수 있으나, 일반적으로 고객 이탈률(1년 동안 이탈한 고객의 비율)을 사용하여 계산할 수 있습니다. 고객 평균 수명은 1을 고객 이탈률로 나눈 값으로 계산됩니다.

평균 구매 가치는 sale_price.sum() / frequency 한 것 쓰면 될듯(=기존 RFM구해줄 때 M과 동일) (고객별로 값을 가지게 될듯)   
평균 구매 빈도는 총 구매 횟수 / 구매한 고객 총수 (전부 공통적인 상수 값을 쓰게될듯)     
고객 평균 수명은 고객 이탈률 (1년 동안 이탈한 고객의 비율) 구해서 역수 취한 것

In [4]:
order_items_extracted.head(3)

,order_item_id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaT,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaT,NaT,0.02
5,154022,106052,84693,14235,415780,Shipped,2023-09-19 09:13:08+00:00,2023-09-16 09:24:00+00:00,NaT,NaT,0.02


In [5]:
order_items_extracted['order_id'].nunique() #총 구매 갯수

80858

In [6]:
order_items_extracted['user_id'].nunique() #총 구매한 고객 수

58464

In [7]:
order_items_extracted['order_id'].nunique() / order_items_extracted['user_id'].nunique() #평균 구매 빈도

1.3830391351943077

In [8]:
avg_purchase_frequency = order_items_extracted['order_id'].nunique() / order_items_extracted['user_id'].nunique()

In [9]:
clv = order_items_extracted.groupby('user_id').agg(
    recency=('created_at', lambda date: (use_standard_date - date.max()).days), 
    frequency=('order_id','nunique'), 
    monetary=('sale_price', 'sum')
)
clv['monetary'] = clv['monetary'] / clv['frequency']
clv

,recency,frequency,monetary
user_id,,,
1,498,1,159.990000
2,648,1,22.000000
3,114,4,100.550000
4,408,1,148.000000
6,598,1,144.820004
...,...,...,...
99989,90,1,15.120000
99992,478,1,55.950001
99996,260,2,61.935000


In [10]:
len(clv)/len(clv[clv['recency']>365])

2.29630793401414

In [11]:
avg_life_time = len(clv)/len(clv[clv['recency']>365])

In [12]:
clv = clv.assign(clv=clv['monetary'] * avg_purchase_frequency * avg_life_time)

In [13]:
clv

,recency,frequency,monetary,clv
user_id,,,,
1,498,1,159.990000,508.109639
2,648,1,22.000000,69.869442
3,114,4,100.550000,319.335111
4,408,1,148.000000,470.030793
6,598,1,144.820004,459.931494
...,...,...,...,...
99989,90,1,15.120000,48.019363
99992,478,1,55.950001,177.690698
99996,260,2,61.935000,196.698361
